In [2]:
import requests
import util.item_analysis as ui
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

# 예
market = ['심리 상담 센터']
addr = '서울시 영등포구'

# 고객이 입력한 상품을 파는 매장을 리스트로 가져오기
market_list = market

In [3]:
# 고객이 입력한 주소 ['시', '구'] 또는 ['군', '면', '리'] 가져오기
addr_list = addr.split(' ')
addr_list1 = []
addr_list2 = []
for i in addr_list:
    if i[-1] == '시':
        addr_list1.append(i)
    elif i[-1] == '구':
        addr_list1.append(i)
    elif i[-1] == '군':
        addr_list2.append(i)
    elif i[-1] == '면':
        addr_list2.append(i)
    elif i[-1] == '리':
        addr_list2.append(i)

addr1 = ' '.join(addr_list1)
addr2 = ' '.join(addr_list2)

if addr1 != '':
    addr = addr1
else:
    addr = addr2
addr

'서울시 영등포구'

In [4]:
# 고객이 입력한 주소 좌표 구하기
try:
    with open('keys/카카오api.txt') as file:
        kakao_key = file.read()

    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(addr)}'
    result = requests.get(url, headers=header).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
except:
    print('주소 형식이 올바르지 않습니다.')

In [5]:
lat, lng

(37.5263635705036, 126.896273923703)

In [6]:
# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

In [7]:
# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

In [8]:
# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)

In [9]:
# '상세 주소 화살표' 클릭
try:
    juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
    juso_way[0].click()
    time.sleep(1)
except:
    print('근처에 매장이 없습니다')

In [10]:
# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source)
market_list = soup.find('ul', class_='eDFz9')
mart = market_list.find('li', recursive=False)
mart

<li class="VLTHu" data-laim-exp-id="undefined"><div class="qbGlu"><div class="ouxiq"><a class="P7gyV" href="https://m.place.naver.com/place/1069666803?entry=ple" role="button"><div class="ApCpt"><div class="place_bluelink C6RjW"><span class="YwYLL">닥터앤맘</span><span class="YzBgS">아동,청소년상담</span></div></div></a><a class="P7gyV" href="https://m.place.naver.com/place/1069666803?entry=ple" role="button"><div class="d7iiF KeRrO"><div class="FTiLM"><span class="XP3ml yTY83">영업 중</span><span class="XP3ml">18:00에 영업 종료</span></div></div></a><div class="d7iiF"><div class="KgfA6"><span class="lWwyx NVngW"><span class="place_blind">현재 위치에서</span>220m</span><span class="lWwyx"><a aria-expanded="true" aria-haspopup="true" class="uFxr1" href="#" role="button"><span class="Pb4bU">서울 영등포구 당산동3가</span><span class="_44_8"><span class="place_blind">상세주소 닫기</span><svg aria-hidden="true" class="Y2sSu" viewbox="0 0 13 8" xmlns="http://www.w3.org/2000/svg"><path d="M6.5 7.7L.1 1.4.8.7l5.6 5.6L12 .7l.7.7-6.2 6

In [11]:
results = []
# 현재 위치에서 마켓 거리
m_distance = mart.select_one('span.lWwyx.NVngW').get_text().split('서')[1]
# 마켓 이름
m_title = mart.select_one('span.place_bluelink.YwYLL').get_text()
# 마켓 도로명 주소
if mart.select_one('div.zZfO1').get_text()[:2] == '지번':
    m_addr = mart.select_one('div.zZfO1').get_text()[2:-2]
else:
    m_addr = mart.select_one('div.zZfO1').get_text()[3:-2]
# 딕셔너리 형태로 저장
market_data = {
    '거리': m_distance,
    '매장명': m_title,
    '주소': m_addr
}
# 딕셔너리 형태로 저장한 것을 리스트에 저장 (3가지 마켓을 넣어야 하므로)
results.append(market_data)
results

AttributeError: 'NoneType' object has no attribute 'get_text'